# 1. Prepare Environment

In [61]:
# import the necessary module 
import time
from pyspark.ml.feature import Word2Vec,HashingTF,IDF,Tokenizer
from pyspark.sql.functions import udf
from pyspark.sql.types import StructType, StructField, StringType,IntegerType, FloatType
from pyspark.sql import SparkSession
import tensorflow_hub as hub
import tensorflow as tf
import numpy as np
import nltk
import re
tokenizer = nltk.data.load('/home/hadoop/nltk_data/tokenizers/punkt/english.pickle')

VBox()

In [62]:
spark = SparkSession \
    .builder \
    .appName("Assignment2") \
    .getOrCreate()

VBox()

# 2. ml_utils

In [63]:
# design necessary methods that will be used in Stage1, Stage2, Stage3, Stage4
store_route = "s3://5349a2/"
module_url = "https://tfhub.dev/google/universal-sentence-encoder/2" #@param ["https://tfhub.dev/google/universal-sentence-encoder/2", "https://tfhub.dev/google/universal-sentence-encoder-large/3"]
embed = hub.Module(module_url)
def getmediumnumber(dataset,length,colname):
    """
    get the median number
    """
    median_num_index = []
    median_num = 0
    if length%2 ==0:
        median_num_index = [length/2-1,length/2]
    else:
        median_num_index = [length/2]
    for i in median_num_index:
        median_num += dataset[int(i)][colname]
    median_num = median_num/float(len(median_num_index))*(-1)
    return median_num

def sentence_token_nltk(string):
    """
    use the method from nltk to segment this paragraph to sentences
    """
    sent_tokenize_list = tokenizer.tokenize(string)
    return sent_tokenize_list
def sentence_len(str_centence):
    """
    calculate the amount of sentences of the review body
    """    
    if str_centence == None:
        return 0
    str_centence = str(str_centence)
    list_ret = tokenizer.tokenize(str_centence)
    return len(list_ret)
def dividesentences(record):
    """
    segment this paragraph to sentences
    """
    sentences = sentence_token_nltk(record[-1])
    reviewid = record[0]
    sentencelist = []
    for sentence in sentences:
        sentencelist.append((reviewid,sentence))
    return sentencelist
def review_embed(sentence_list):
    """
    use google methods embedding sentence
    """   
    with tf.Session() as session:
        session.run([tf.global_variables_initializer(), tf.tables_initializer()])
        message_embeddings = session.run(embed(sentence_list))
    return message_embeddings
def googleembeddingmethod(text_rdd):
    
    review_embedding = text_rdd.mapPartitions(review_embed).collect()
    return review_embedding 
# def cosdistance(A,B):
#     num = float(np.dot(A * B.T)) 
#     denom = np.linalg.norm(A) * np.linalg.norm(B)  
#     cos = num / denom 
#     return (1-cos)



VBox()

# 3. Getting The necessary data  

In [64]:
# rating_data_route = "s3://5349a2/test_Music.tsv"
rating_data_route = "s3://amazon-reviews-pds/tsv/amazon_reviews_us_Music_v1_00.tsv.gz"
music_data_df = spark.read.csv(rating_data_route,header=True,sep='\t')\
            .select("customer_id","product_id","star_rating","review_id","review_body").na.drop()

# write the result to file 
music_data_df.write.csv(store_route+'music_data_df.csv',header = True,mode = "overwrite")






VBox()

# Stage 1. overall review

Stage1 --- (a)  total number of reviews, user, and product

In [97]:
t1 = time.time()
# read the selected music data from s3 
music_data_df = spark.read.csv(store_route+'music_data_df.csv', header=True, inferSchema=True).cache()

#total number of reviews
total_number_of_reviews= music_data_df.select("review_id").count()
#total number of unique user
total_number_unique_user = music_data_df.select("customer_id").distinct().count()
#total number of unique product
total_number_unique_product = music_data_df.select("product_id").distinct().count()


# print(total_number_of_reviews)
# print(total_number_unique_user)
# print(total_number_unique_product)

VBox()

Stage1 --- (b)  Top 10, largest number and medium number of reviews created by costomer

In [98]:

number_of_reviews_by_user_pre = music_data_df.select("customer_id").groupby("customer_id").count()
number_of_reviews_by_user=number_of_reviews_by_user_pre.select(number_of_reviews_by_user_pre["customer_id"],number_of_reviews_by_user_pre["count"]*(-1))
number_of_reviews_by_user_list = number_of_reviews_by_user.orderBy("(count * -1)").limit(int(total_number_unique_user/2+1)).collect()

#median number of customer
median_num_user = getmediumnumber(number_of_reviews_by_user_list,total_number_unique_user,"(count * -1)")
#top 10 number of reviews by user
top10_number_of_reviews_by_user =[number_of_reviews_by_user_list[row_index]["customer_id"] for row_index in range(10)]
#largest number of reviews by user
largest_number_of_reviews_by_user = number_of_reviews_by_user_list[0]["(count * -1)"]*(-1)

# print(top10_number_of_reviews_by_user)
# print(largest_number_of_reviews_by_user)
# print(median_num_user)



VBox()

Stage1 --- (c)  Top 10, largest number and medium number of reviews created by costomer

In [99]:
number_of_reviews_for_single_product_pre = music_data_df.select("product_id").groupby("product_id").count()
number_of_reviews_for_single_product=number_of_reviews_for_single_product_pre.select(number_of_reviews_for_single_product_pre["product_id"],number_of_reviews_for_single_product_pre["count"]*(-1)).orderBy("(count * -1)").limit(int(total_number_unique_product/2+1))
number_of_reviews_for_single_product_list = number_of_reviews_for_single_product.collect()

#median number of customer
median_num_product = getmediumnumber(number_of_reviews_for_single_product_list,total_number_unique_product,"(count * -1)")

#top 10 number of reviews for a product
top10_number_of_reviews_for_product =[number_of_reviews_for_single_product_list[row_index]["product_id"] for row_index in range(10)]
#largest number of reviews for a user
largest_number_of_reviews_for_product = number_of_reviews_for_single_product_list[0]["(count * -1)"]*(-1)


# print(top10_number_of_reviews_for_product)
# print(largest_number_of_reviews_for_product)
# print(median_num_product)

# write this result to S3 because this filtered dateframe will be used in the later processing
number_of_reviews_for_single_product.limit(10).write.csv(store_route+'number_of_reviews_for_single_product.csv', header = True, mode = "overwrite")
t2 = time.time()
print(t2-t1)





VBox()

57.30673003196716

In [100]:
number_of_reviews_by_user = music_data_df.select("customer_id").groupby("customer_id").count()
userid_over_median = number_of_reviews_by_user.filter(number_of_reviews_by_user["count"]>median_num_user)

number_of_reviews_for_single_product = music_data_df.select("product_id","customer_id").groupby("product_id").count()
productid_over_median = number_of_reviews_for_single_product.filter(number_of_reviews_for_single_product["count"]>median_num_product)

# write this userid_over_median dataframe to S3 because this filtered dateframe will be used in the later processing
userid_over_median.write.csv(store_route+'userid_over_median.csv', header = True, mode = "overwrite")
productid_over_median.write.csv(store_route+'productid_over_median.csv', header = True, mode = "overwrite")

VBox()

# Stage 2. remove nnwanted rows

In [101]:
t1 = time.time()
music_data_df = spark.read.csv(store_route+'music_data_df.csv', header=True, inferSchema=True).cache()


userid_over_median = spark.read.csv(store_route+'userid_over_median.csv', header=True, inferSchema=True)
productid_over_median = spark.read.csv(store_route+'productid_over_median.csv', header=True, inferSchema=True)


toDateUDF=udf(sentence_len, IntegerType())
review_body_len = music_data_df.withColumn('s_len',toDateUDF('review_body'))
review_body_over_2_sentence = review_body_len.filter(review_body_len["s_len"]>=2)




VBox()

In [102]:
#use inner join to get the filtered data
filter_movie_data = review_body_over_2_sentence.join(productid_over_median,"product_id","inner").select("customer_id","product_id","star_rating","review_id","review_body","s_len")\
.join(userid_over_median,"customer_id","inner").select("customer_id","product_id","star_rating","review_id","review_body","s_len")

filter_movie_data2 = filter_movie_data.select("product_id","review_id","star_rating","review_body")
#this data will be used in later processing
filter_movie_data2.write.csv(store_route+'filter_movie_data.csv', header = True, mode = "overwrite")



VBox()

In [103]:
filter_movie_data_rdd = filter_movie_data.rdd.cache()


VBox()

In [104]:
reviews_by_user = filter_movie_data_rdd.map(lambda x:(x[0],x[5])).groupByKey()
reviews_by_user_sort = reviews_by_user.mapValues(lambda l : sorted(l,key = lambda x : x)[int(len(l)/2)])
reviews_by_user_sort_10 = reviews_by_user_sort.sortBy(lambda x:x[-1],False).take(10)


reviews_by_prod = filter_movie_data_rdd.map(lambda x:(x[1],x[5])).groupByKey()

reviews_by_prod_sort = reviews_by_prod.mapValues(lambda l : sorted(l,key = lambda x : x)[int(len(l)/2)])
reviews_by_prod_sort_10 = reviews_by_prod_sort.sortBy(lambda x:x[-1],False).take(10)



print(reviews_by_user_sort_10)

print(reviews_by_prod_sort_10)
t2 = time.time()
print(t2-t1)


VBox()

[(51865782, 440), (25628286, 251), (42072921, 243), (46097534, 228), (37118941, 227), (29580246, 201), (50476169, 200), (21809895, 194), (50595705, 191), (10794201, 185)]
[('B00LTQ5EVY', 984), ('B00T7TYTCK', 503), ('B009SF2GZU', 321), ('B009SF2IRG', 308), ('B005ZHBBU6', 293), ('B0000020FQ', 270), ('B0002IJNGC', 268), ('B000RY431G', 267), ('B000003G29', 267), ('B00IOQSW7A', 256)]
1734.204293012619

# Stage3. Similarity analysis with Sentence Embedding

In [105]:
from pyspark.sql import SparkSession
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np


VBox()

### Stage3 --- (a) pick one of top 10 product

In [106]:
t1 = time.time()
number_of_reviews_for_single_product_10 = spark.read.csv(store_route+'number_of_reviews_for_single_product.csv', header=True, inferSchema=True)
pickproduct_id = number_of_reviews_for_single_product_10.collect()[-1]["product_id"]
print(pickproduct_id)
# this dataframe won't be used again, so delete it, release memory
del number_of_reviews_for_single_product_10

VBox()

B00006J6VG

In [107]:
filter_movie_data_rdd = spark.read.csv(store_route+'filter_movie_data.csv', header=True, inferSchema=True).rdd
# review_star will be used multiple times so cache it
review_star = filter_movie_data_rdd.filter(lambda l : l[0] == pickproduct_id).map(lambda l : (l[1],l[2],l[3])).cache()
# this dataframe won't be used again, so delete it, release memory
del filter_movie_data_rdd

VBox()

### get negative and positive reviews cluster



In [108]:
#get negative and positive reviews cluster
negative_reviews = review_star.filter(lambda l: l[1]<=2)
positive_reviews = review_star.filter(lambda l: l[1]>=4)
# print(negative_reviews.collect())


VBox()

In [109]:
negative_reviews_divide = negative_reviews.flatMap(dividesentences).map(lambda l : (l[0],re.sub(r'</?\w+[^>]*>','',l[-1]))).cache()
positive_reviews_divide = positive_reviews.flatMap(dividesentences).map(lambda l : (l[0],re.sub(r'</?\w+[^>]*>','',l[-1]))).cache()


VBox()

In [110]:
from pyspark.sql import Row

negative_reviews_divide_df = negative_reviews_divide.map(lambda l : Row(review_id= l[0],reviews_body = l[1])).toDF()
positive_reviews_divide_df = positive_reviews_divide.map(lambda l : Row(review_id= l[0],reviews_body = l[1])).toDF()

negative_reviews_divide_df.write.csv(store_route+'negative_reviews_divide_df.csv', header = True, mode = "overwrite")
positive_reviews_divide_df.write.csv(store_route+'positive_reviews_divide_df.csv', header = True, mode = "overwrite")




VBox()

In [111]:
def getcenterpoint(reviews_divide,modelname):
    print("===================================Start===================================")

    print(modelname+" center cluster : ---------------------------------------------------------")
    #get the sentence list 
    text_list = reviews_divide.map(lambda l : l[1]).collect()
    text_rdd = reviews_divide.map(lambda row: str(row[-1])).filter(lambda data: data is not None).cache()
    
    
    #get the review id list 
    reviewid_list = reviews_divide.map(lambda l : l[0]).collect()
    
    review_embedding_list = np.array(review_embed(text_rdd.collect()))


    average_distance = []
    review_embedding_array = np.array(review_embedding_list)
    review_embedding_array_norm = (review_embedding_array**2).sum(axis=1)**0.5

    #use matrix to calculate distance
    #convert the original method(1-cos(Theta)) to 
    #avg_distance = len(vector_list)-vector.dot(sum(direction of all vector))
    for i in range(np.shape(review_embedding_array)[0]):
        line = np.array([review_embedding_array[i]])
        line_dis = (np.sum(line**2)**0.5)*review_embedding_array_norm**(-1)
        dot_value = np.dot(review_embedding_array,line.T)
        average_dis = (np.shape(review_embedding_array)[0] - np.dot(dot_value.T,line_dis).sum())/np.shape(review_embedding_array)[0]
        average_distance.append(average_dis)
        
    #get the center_id_index -- the minimum 
    center_id_index = average_distance.index(np.min(average_distance))
    
    
    #get the center_id by center_id_index
    center_id = reviewid_list[center_id_index]
    
    
    #get the center_vec to calculate the distance with other sentence 
    center_vec = np.array([review_embedding_array[center_id_index]])
    print("center review id:"+str(center_id))
    print("center sentence:"+str(text_list[center_id_index]))

    cos_dis = np.dot(review_embedding_array,center_vec.T)
    line_dis = (np.sum(center_vec**2)**0.5)*review_embedding_array_norm
    center_dis = []
    for i in range(np.shape(line_dis)[0]):
        center_dis.append(1-(cos_dis[i][0]/line_dis[i]))
    
    
    #get the index of cloestest 10 sentences  
    cloest_10 = np.array(center_dis).argsort()[1:11]
    for i in cloest_10:
        print("review id:" + str(reviewid_list[i]))
        print("sentence:"+str(text_list[i]))

    print("====================================End====================================")




VBox()

In [112]:
getcenterpoint(positive_reviews_divide,"positive")
getcenterpoint(negative_reviews_divide,"negative")
t2 = time.time()

print(t2-t1)


VBox()

===================================Start===================================
positive center cluster : ---------------------------------------------------------
center review id:R1VR2BOC4IT29H
center sentence:Every song is great!
review id:R2L30H6HKZXUK7
sentence:every song is really good.
review id:R144NZND4C5S5A
sentence:Every single song is GREAT!
review id:R2N0NUDL7GEOCN
sentence:i LOVE every single song.
review id:R22YSMYT6ZBKWH
sentence:all the songs are great.
review id:RK0YKYW86PK2N
sentence:all the songs are great.
review id:R1ZAV0KB4C8FY5
sentence:The songs are all great!
review id:R32U2AEZWXU5X0
sentence:Almost all the songs on here are great.
review id:R71OTQFQVOIVL
sentence:The BEST SONG ON THE ALBLUM IS HOLD ON!
review id:R2H1YNHUCT31TS
sentence:It's got a great variation of songs, and every one of them is worth listening to.
review id:R1GQBYL3HBOYSZ
sentence:I love every song in this C.D.
====================================End====================================
========

# Stage4. Similarity analysis with Spark supported Feature Extractors

In [113]:
def Pca(X,k=21):
    '''
    Pca decompose: reduce the calculation dimension, default dimension is 21
    '''
    n_samples, n_features = X.shape
    mean=np.array([np.mean(X[:,i]) for i in range(n_features)])
    norm_X=X-mean
    scatter_matrix=np.dot(np.transpose(norm_X),norm_X)
    eig_val, eig_vec = np.linalg.eig(scatter_matrix)
    eig_pairs = [(np.abs(eig_val[i]), eig_vec[:,i]) for i in range(n_features)]
    eig_pairs.sort(reverse=True)
    feature=np.array([ele[1] for ele in eig_pairs[:k]])
    data=np.dot(norm_X,np.transpose(feature))
    return data

def Norm(raw_data):
    '''
    nomalization : convert the value to 0-1 
    '''
    minVals = raw_data.min(0)
    maxVals = raw_data.max(0)

    ranges = maxVals - minVals
    normDataSet = np.zeros(np.shape(raw_data))
    m = raw_data.shape[0]
    normDataSet = raw_data - np.tile(minVals, (m, 1))
    normDataSet = normDataSet/np.tile(ranges, (m, 1))
    return normDataSet

VBox()

In [114]:
t1 = time.time()
#get negative_reviews and positive_reviews from S3,these data has been stored in Stage3
negative_reviews_divide_df = spark.read.csv(store_route+'negative_reviews_divide_df.csv', header=True, inferSchema=True)
positive_reviews_divide_df = spark.read.csv(store_route+'positive_reviews_divide_df.csv', header=True, inferSchema=True)


negative_reviews_divide = negative_reviews_divide_df.rdd
positive_reviews_divide = positive_reviews_divide_df.rdd

negative_text_rdd = negative_reviews_divide.map(lambda row: str(row[-1])).filter(lambda data: data is not None).cache()
positive_text_rdd = positive_reviews_divide.map(lambda row: str(row[-1])).filter(lambda data: data is not None).cache()

negative_text_list = negative_reviews_divide.map(lambda l : l[1]).collect()
positive_text_list = positive_reviews_divide.map(lambda l : l[1]).collect()


VBox()

In [115]:
### Create training model

VBox()

In [116]:
from pyspark.ml.feature import Word2Vec,HashingTF,IDF,Tokenizer
from nltk.tokenize import WordPunctTokenizer
word_tokenizer = WordPunctTokenizer()
# pattern = r"\w+(?:[-']\w+)*|'|[-.(]+|\S\w*"


#segement the sentence to words
negative_word_list = [(word_tokenizer.tokenize(str(sentence)),) for sentence in negative_text_list]
positive_word_list = [(word_tokenizer.tokenize(str(sentence)),) for sentence in positive_text_list]
# transform to dataframe
negative_documentDF = spark.createDataFrame(negative_word_list, ["text"])
positive_documentDF = spark.createDataFrame(positive_word_list, ["text"])



word2Vec = Word2Vec(vectorSize=50, minCount=0, inputCol="text", outputCol="result")

negative_model = word2Vec.fit(negative_documentDF)
positive_model = word2Vec.fit(positive_documentDF)

modeldict = {"positive":positive_model,"negative":negative_model}
dataframedict = {"positive":positive_documentDF,"negative":negative_documentDF}

VBox()

In [117]:
# use trained model to get trained result
def spl_word_vec_embedding(modelchoice):
    current_model = modeldict[modelchoice]
    dataDF = dataframedict[modelchoice]
    embeddingresult = current_model.transform(dataDF).select("result").collect()
    embedding_array =np.array([i["result"] for i in embeddingresult])
    return embedding_array



# print(np.shape(a))
# print(np.shape(b))
# print(np.shape(c))
# print(c)

VBox()

In [118]:
def getcenterand_toptenpoint_word2vec(reviews_divide,modelname):
    print("====================================Start====================================")

    print(modelname+" center cluster : ---------------------------------------------------------")
    text_list = reviews_divide.map(lambda l : l[1]).collect()
    print
    text_rdd = reviews_divide.map(lambda row: str(row[-1])).filter(lambda data: data is not None).cache()
    reviewid_list = reviews_divide.map(lambda l : l[0]).collect()


    review_embedding_array = Pca(spl_word_vec_embedding(modelname))
    average_distance = []
#     print(np.shape(review_embedding_array))
    review_embedding_array_norm = np.array([(review_embedding_array**2).sum(axis=1)**0.5]).T
    
    
    #use matrix to calculate distance
    #convert the original method(1-cos(Theta)) to 
    #avg_distance = len(vector_list)-vector.dot(sum(direction of all vector))
    for i in range(np.shape(review_embedding_array)[0]):
        line = np.array([review_embedding_array[i]])
        line_dis = (np.sum(line**2)**0.5)*review_embedding_array_norm**(-1)
        dot_value = np.dot(review_embedding_array,line.T)
        average_dis = (np.shape(review_embedding_array)[0] - (dot_value/line_dis).sum())/np.shape(review_embedding_array)[0]
        average_distance.append(average_dis)
        
    #get the center_id by center_id_index
    center_id_index = average_distance.index(min(average_distance))
    center_id = reviewid_list[center_id_index]
#     print(center_id)

    #get the center_vec to calculate the distance with other sentence 
    center_vec = np.array([review_embedding_array[center_id_index]])
    print("center review id:"+str(center_id))
    print("center sentence:"+str(text_list[center_id_index]))

    cos_dis = np.dot(review_embedding_array,center_vec.T)
    line_dis = (np.sum(center_vec**2)**0.5)*review_embedding_array_norm
    center_dis = []
    
    #get the index of cloestest 10 sentences  
    for i in range(np.shape(line_dis)[0]):
        center_dis.append(1-(cos_dis[i]/line_dis[i]))
    center_dis = [i[0] for i in center_dis]
    cloest_10 = np.array(center_dis).argsort()[1:11]
    for i in cloest_10:
        print("review id:" + str(reviewid_list[i]))
        print("sentence:"+str(text_list[i]))
    print("====================================End====================================")



VBox()

In [119]:
getcenterand_toptenpoint_word2vec(positive_reviews_divide,"positive")
getcenterand_toptenpoint_word2vec(negative_reviews_divide,"negative")

t2 = time.time()

print(t2-t1)


VBox()

====================================Start====================================
positive center cluster : ---------------------------------------------------------
center review id:R2EKY5I5KJW2PB
center sentence:6.Girls & Boys-5/5-Awesome song!
review id:R26OJ99Q1J1B6Q
sentence:Riot Girl - 5/510.
review id:RY3PCDQ80U0O8
sentence:My Bloody Valentine: 5/5.
review id:RY3PCDQ80U0O8
sentence:Riot Girl: 3/5.
review id:R2W54S6JP18GBZ
sentence:My Bloody Valentine 5/5    8.
review id:RY3PCDQ80U0O8
sentence:Hold On: 5/5.
review id:R26OJ99Q1J1B6Q
sentence:My Bloody Valentine - 4/58.
review id:R26OJ99Q1J1B6Q
sentence:Hold On - 5/59.
review id:RPRNHKLZG1OEY
sentence:Riot Girl~ 4/5 Pretty fast paced.
review id:RY3PCDQ80U0O8
sentence:Girls and Boys: 4/5.
review id:R9C2ZW7ODCQRT
sentence:Riot Girl 4.
====================================End====================================
====================================Start====================================
negative center cluster : --------------------------